# Brain tumor 3D segmentation with MONAI

This tutorial shows how to construct a training workflow of multi-labels segmentation task.  
And it contains below features:
1. Transforms for dictionary format data.
2. Define a new transform according to MONAI transform API.
3. Load Nifti image with metadata, load a list of images and stack them.
4. Randomly adjust intensity for data augmentation.
5. Cache IO and transforms to accelerate training and validation.
6. 3D UNet model, Dice loss function, Mean Dice metric for 3D segmentation task.
7. Deterministic training for reproducibility.

The Brain tumor dataset can be downloaded from http://medicaldecathlon.com/.  
Target: Gliomas segmentation necrotic/active tumour and oedema  
Modality: Multimodal multisite MRI data (FLAIR, T1w, T1gd,T2w)  
Size: 750 4D volumes (484 Training + 266 Testing)  
Source: BRATS 2016 and 2017 datasets.  
Challenge: Complex and heterogeneously-located targets

Below figure shows image patches with the tumor sub-regions that are annotated in the different modalities (top left) and the final labels for the whole dataset (right). (Figure taken from the [BraTS IEEE TMI paper](https://ieeexplore.ieee.org/document/6975210/))  
![image](./images/brats_tasks.png)
The image patches show from left to right:
1. the whole tumor (yellow) visible in T2-FLAIR (Fig.A).
2. the tumor core (red) visible in T2 (Fig.B).
3. the enhancing tumor structures (light blue) visible in T1Gd, surrounding the cystic/necrotic components of the core (green) (Fig. C).
4. The segmentations are combined to generate the final labels of the tumor sub-regions (Fig.D): edema (yellow), non-enhancing solid core (red), necrotic/cystic core (green), enhancing core (blue).

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import glob
import numpy as np
import torch
import matplotlib.pyplot as plt
from monai.transforms import \
    MapTransform, Compose, LoadNiftid, AsChannelFirstd, Spacingd, Orientationd, \
    RandSpatialCropd, RandFlipd, NormalizeIntensityd, RandScaleIntensityd, \
    RandShiftIntensityd, CenterSpatialCropd, ToTensord
from monai.data import DataLoader
from monai.apps import DecathlonDataset
from monai.inferers import sliding_window_inference
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.utils import set_determinism

## Set deterministic training for reproducibility

In [ ]:
set_determinism(seed=0)

## Define a new transform to convert brain tumor labels
Here we convert the multi-classes labels into multi-labels segmentation task in One-Hot format.

In [ ]:
class ConvertToMultiChannelBasedOnBratsClassesd(MapTransform):
    """
    Convert labels to multi channels based on brats classes:
    label 1 is the peritumoral edema
    label 2 is the GD-enhancing tumor
    label 3 is the necrotic and non-enhancing tumor core
    The possible classes are TC (Tumor core), WC (Whole tumor)
    and ET (Enhancing tumor).

    """
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = list()
            # merge label 2 and label 3 to construct TC
            result.append(np.logical_or(d[key] == 2, d[key] == 3))
            # merge labels 1, 2 and 3 to construct WC
            result.append(np.logical_or(np.logical_or(d[key] == 2, d[key] == 3), d[key] == 1))
            # label 2 is ET
            result.append(d[key] == 2)
            d[key] = np.stack(result, axis=0).astype(np.float32)
        return d

## Setup transforms for training and validation

In [ ]:
train_transform = Compose([
    # load 4 Nifti images and stack them together
    LoadNiftid(keys=['image', 'label']),
    AsChannelFirstd(keys='image'),
    ConvertToMultiChannelBasedOnBratsClassesd(keys='label'),
    Spacingd(keys=['image', 'label'], pixdim=(1.5, 1.5, 2.), mode=('bilinear', 'nearest')),
    Orientationd(keys=['image', 'label'], axcodes='RAS'),
    RandSpatialCropd(keys=['image', 'label'], roi_size=[128, 128, 64], random_size=False),
    RandFlipd(keys=['image', 'label'], prob=0.5, spatial_axis=0),
    NormalizeIntensityd(keys='image', nonzero=True, channel_wise=True),
    RandScaleIntensityd(keys='image', factors=0.1, prob=0.5),
    RandShiftIntensityd(keys='image', offsets=0.1, prob=0.5),
    ToTensord(keys=['image', 'label'])
])
val_transform = Compose([
    LoadNiftid(keys=['image', 'label']),
    AsChannelFirstd(keys='image'),
    ConvertToMultiChannelBasedOnBratsClassesd(keys='label'),
    Spacingd(keys=['image', 'label'], pixdim=(1.5, 1.5, 2.), mode=('bilinear', 'nearest')),
    Orientationd(keys=['image', 'label'], axcodes='RAS'),
    CenterSpatialCropd(keys=['image', 'label'], roi_size=[128, 128, 64]),
    NormalizeIntensityd(keys='image', nonzero=True, channel_wise=True),
    ToTensord(keys=['image', 'label'])
])

## Quickly load data with DecathlonDataset
Here we use `DecathlonDataset` to automatically download and extract the dataset.
It inherits MONAI `CacheDataset`, so we set `cache_num=100` to cache 100 items for training and use the defaut args to cache all the items for validation.

In [ ]:
root_dir = '/workspace/data/medical'
train_ds = DecathlonDataset(root_dir=root_dir, task='Task01_BrainTumour', transform=train_transform,
                            section='training', download=True, num_workers=4, cache_num=100)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4)
val_ds = DecathlonDataset(root_dir=root_dir, task='Task01_BrainTumour', transform=val_transform,
                          section='validation', download=False, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=4)

## Check data shape and visualize

In [ ]:
# pick one image from DecathlonDataset to visualize and check the 4 channels
print(f"image shape: {val_ds[9]['image'].shape}")
plt.figure('image', (24, 6))
for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.title(f"image channel {str(i)}")
    plt.imshow(val_ds[9]['image'][i, :, :, 20].detach().cpu(), cmap='gray')
plt.show()
# also visualize the 3 channels label corresponding to this image
print(f"label shape: {val_ds[9]['label'].shape}")
plt.figure('label', (18, 6))
for i in range(3):
    plt.subplot(1, 3, i + 1)
    plt.title(f"label channel {str(i)}")
    plt.imshow(val_ds[9]['label'][i, :, :, 20].detach().cpu())
plt.show()

## Create Model, Loss, Optimizer

In [ ]:
# standard PyTorch program style: create UNet, DiceLoss and Adam optimizer
device = torch.device('cuda:0')
model = UNet(dimensions=3, in_channels=4, out_channels=3, channels=(16, 32, 64, 128, 256),
             strides=(2, 2, 2, 2), num_res_units=2).to(device)
loss_function = DiceLoss(to_onehot_y=False, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5, amsgrad=True)

## Execute a typical PyTorch training process

In [ ]:
epoch_num = 180
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()
metric_values_tc = list()
metric_values_wt = list()
metric_values_et = list()
for epoch in range(epoch_num):
    print('-' * 10)
    print(f"epoch {epoch + 1}/{epoch_num}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data['image'].to(device), batch_data['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f"{step}/{len(train_ds) // train_loader.batch_size}, train_loss: {loss.item():.4f}")
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            dice_metric = DiceMetric(include_background=True, sigmoid=True, reduction='mean')
            metric_sum = metric_sum_tc = metric_sum_wt = metric_sum_et = 0.
            metric_count = metric_count_tc = metric_count_wt = metric_count_et = 0
            for val_data in val_loader:
                val_inputs, val_labels = val_data['image'].to(device), val_data['label'].to(device)
                val_outputs = model(val_inputs)
                # compute overall mean dice
                value = dice_metric(y_pred=val_outputs, y=val_labels)
                not_nans = dice_metric.not_nans.item()
                metric_count += not_nans
                metric_sum += value.item() * not_nans
                # compute mean dice for TC
                value_tc = dice_metric(y_pred=val_outputs[:, 0 : 1], y=val_labels[:, 0 : 1])
                not_nans = dice_metric.not_nans.item()
                metric_count_tc += not_nans
                metric_sum_tc += value_tc.item() * not_nans
                # compute mean dice for WT
                value_wt = dice_metric(y_pred=val_outputs[:, 1 : 2], y=val_labels[:, 1 : 2])
                not_nans = dice_metric.not_nans.item()
                metric_count_wt += not_nans
                metric_sum_wt += value_wt.item() * not_nans
                # compute mean dice for ET
                value_et = dice_metric(y_pred=val_outputs[:, 2 : 3], y=val_labels[:, 2 : 3])
                not_nans = dice_metric.not_nans.item()
                metric_count_et += not_nans
                metric_sum_et += value_et.item() * not_nans
                
            metric = metric_sum / metric_count
            metric_values.append(metric)
            metric_tc = metric_sum_tc / metric_count_tc
            metric_values_tc.append(metric_tc)
            metric_wt = metric_sum_wt / metric_count_wt
            metric_values_wt.append(metric_wt)
            metric_et = metric_sum_et / metric_count_et
            metric_values_et.append(metric_et)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), 'best_metric_model.pth')
                print('saved new best metric model')
            print(f"current epoch: {epoch + 1} current mean dice: {metric:.4f}"
                  f" tc: {metric_tc:.4f} wt: {metric_wt:.4f} et: {metric_et:.4f}"
                  f"\nbest mean dice: {best_metric:.4f} at epoch: {best_metric_epoch}")

In [ ]:
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

## Plot the loss and metric

In [ ]:
plt.figure('train', (12, 6))
plt.subplot(1, 2, 1)
plt.title('Epoch Average Loss')
x = [i + 1 for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel('epoch')
plt.plot(x, y, color='red')
plt.subplot(1, 2, 2)
plt.title('Val Mean Dice')
x = [val_interval * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel('epoch')
plt.plot(x, y, color='green')
plt.show()

plt.figure('train', (18, 6))
plt.subplot(1, 3, 1)
plt.title('Val Mean Dice TC')
x = [val_interval * (i + 1) for i in range(len(metric_values_tc))]
y = metric_values_tc
plt.xlabel('epoch')
plt.plot(x, y, color='blue')
plt.subplot(1, 3, 2)
plt.title('Val Mean Dice WT')
x = [val_interval * (i + 1) for i in range(len(metric_values_wt))]
y = metric_values_wt
plt.xlabel('epoch')
plt.plot(x, y, color='brown')
plt.subplot(1, 3, 3)
plt.title('Val Mean Dice ET')
x = [val_interval * (i + 1) for i in range(len(metric_values_et))]
y = metric_values_et
plt.xlabel('epoch')
plt.plot(x, y, color='purple')
plt.show()

## Check best model output with the input image and label

In [ ]:
model.load_state_dict(torch.load('best_metric_model.pth'))
model.eval()
with torch.no_grad():
    # select one image to evaluate and visualize the model output
    val_input = val_ds[6]['image'].unsqueeze(0).to(device)
    val_output = model(val_input)
    plt.figure('image', (24, 6))
    for i in range(4):
        plt.subplot(1, 4, i + 1)
        plt.title(f"image channel {str(i)}")
        plt.imshow(val_ds[6]['image'][i, :, :, 20].detach().cpu(), cmap='gray')
    plt.show()
    # visualize the 3 channels label corresponding to this image
    plt.figure('label', (18, 6))
    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(f"label channel {str(i)}")
        plt.imshow(val_ds[6]['label'][i, :, :, 20].detach().cpu())
    plt.show()
    # visualize the 3 channels model output corresponding to this image
    plt.figure('output', (18, 6))
    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(f"output channel {str(i)}")
        plt.imshow((val_output[0, i, :, :, 20].sigmoid() >= 0.5).float().detach().cpu())
    plt.show()